# Import Libaries

In [85]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pickle

In [48]:
data = pd.read_feather('output.feather')
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,z axis,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
resume = pd.read_csv('Resume.csv')
y = resume['Category']
data['y'] = y
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION


In [56]:
resume

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [55]:
data['b']

0       0
1       0
2       0
3       0
4       0
       ..
2479    1
2480    0
2481    1
2482    0
2483    0
Name: b, Length: 2484, dtype: int32

# Aggregation by Class

In [50]:
agg_data = data.groupby(['y']).sum()
agg_data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,z axis,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness
y,,,,,,,,,,,,,,,,,,,,,
ACCOUNTANT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADVOCATE,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
AGRICULTURE,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
APPAREL,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ARTS,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AUTOMOBILE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AVIATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BANKING,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
BPO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
agg_data = agg_data.T
# agg_data = agg_data.reset_index()
agg_data

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
abaqus,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
ablation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ableton live,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
abnormal psychology,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
absorption,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoning,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoom,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
zumba,0,0,0,0,1,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0


In [52]:
agg_data.describe()

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
count,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,...,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.000000,5382.00000,5382.000000
mean,0.518395,0.468785,0.302304,0.364363,0.390004,0.145299,0.500557,0.529543,0.117428,0.542549,...,0.470085,0.683203,0.549796,0.409885,0.547566,0.453177,0.773318,0.499628,0.34411,0.298959
std,3.662740,2.677425,1.495651,2.257387,2.149339,0.835290,2.858085,3.137581,0.660819,3.642798,...,2.545869,2.957698,3.699906,2.559233,2.875388,3.064375,3.814298,3.198259,2.53321,2.047747
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,102.000000,100.000000,54.000000,84.000000,79.000000,30.000000,97.000000,106.000000,22.000000,119.000000,...,79.000000,97.000000,118.000000,96.000000,107.000000,104.000000,113.000000,99.000000,94.00000,76.000000


# Identify Significant Skills

We used percentile method here, any value smaller than 90 percentile is not significant. 

In [53]:
# We want to find among all the skills that appear at least once, what is the 90 percentile value for a particular job
print("Accountant:", np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 90))
print("BPO:", np.percentile(agg_data['BPO'][agg_data['BPO'] != 0], 90))

Accountant: 7.100000000000023
BPO: 3.0


In [72]:
# Accountant significant skills
temp = agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] > np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 90)]
temp

administrative support    11
adp payroll                8
auditing                  33
automation                11
b                         20
                          ..
tracking                  13
trial balance             12
use tax                   17
variance analysis         20
wire transfer             12
Name: ACCOUNTANT, Length: 68, dtype: int32

In [73]:
print(temp.max(), temp.min())

102 8


In [74]:
# Scaling
temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min()))

administrative support    0.031915
adp payroll               0.000000
auditing                  0.265957
automation                0.031915
b                         0.127660
                            ...   
tracking                  0.053191
trial balance             0.042553
use tax                   0.095745
variance analysis         0.127660
wire transfer             0.042553
Name: ACCOUNTANT, Length: 68, dtype: float64

In [82]:
# If value > 0.7, skill level required is 5; 
# If 0.7 >= value > 0.3, skill level required is 4;
# If 0.3 >= value, skill level required is 3;
temp2 = temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min())).apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
temp2

administrative support    3
adp payroll               3
auditing                  3
automation                3
b                         3
                         ..
tracking                  3
trial balance             3
use tax                   3
variance analysis         3
wire transfer             3
Name: ACCOUNTANT, Length: 68, dtype: int64

In [84]:
temp2[temp2 == 5]

{'general ledger': 5, 'management': 5, 'reconciliation': 5}

In [47]:
skills_required = {}
for col in agg_data.columns[1:]:
    print(col)
    series = agg_data[col][agg_data[col] > np.percentile(agg_data[col][agg_data[col] != 0], 90)]
    scaled_series = series.apply(lambda x: (x - series.min()) / (series.max() - series.min()))
    binned_series = scaled_series.apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
    skills_required[col] = binned_series.to_dict()

ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT
CHEF
CONSTRUCTION
CONSULTANT
DESIGNER
DIGITAL-MEDIA
ENGINEERING
FINANCE
FITNESS
HEALTHCARE
HR
INFORMATION-TECHNOLOGY
PUBLIC-RELATIONS
SALES
TEACHER


In [ ]:
skills_file = open('skills_required', 'wb')
pickle.dump(skills_required, skills_file)
skills_file.close()

In [ ]:
with open('<directory>', 'rb') as pickle_file:
    content = pickle.load(pickle_file)